In [12]:
import pandas as pd
import plotly.graph_objs as go 
import plotly.offline as py 

In [4]:
#load datasets
df_classes = pd.read_csv("/Users/roy/Documents/UNI/3r/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
df_edges = pd.read_csv("/Users/roy/Documents/UNI/3r/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
df_features = pd.read_csv("/Users/roy/Documents/UNI/3r/elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)

In [5]:
df_features.head

<bound method NDFrame.head of               0    1         2         3         4          5         6    \
0       230425980    1 -0.171469 -0.184668 -1.201369  -0.121970 -0.043875   
1         5530458    1 -0.171484 -0.184668 -1.201369  -0.121970 -0.043875   
2       232022460    1 -0.172107 -0.184668 -1.201369  -0.121970 -0.043875   
3       232438397    1  0.163054  1.963790 -0.646376  12.409294 -0.063725   
4       230460314    1  1.011523 -0.081127 -1.201369   1.153668  0.333276   
...           ...  ...       ...       ...       ...        ...       ...   
203764  173077460   49 -0.145771 -0.163752  0.463609  -0.121970 -0.043875   
203765  158577750   49 -0.165920 -0.123607  1.018602  -0.121970 -0.043875   
203766  158375402   49 -0.172014 -0.078182  1.018602   0.028105 -0.043875   
203767  158654197   49 -0.172842 -0.176622  1.018602  -0.121970 -0.043875   
203768  157597225   49 -0.012037 -0.132276  0.463609  -0.121970 -0.043875   

             7          8         9    ...   

In [6]:
len(df_edges),len(df_features),len(df_classes)

(234355, 203769, 203769)

In [7]:
display(df_edges.head(5),df_features.head(5),df_classes.head(5))

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206


,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117


,txId,class
0,230425980,unknown
1,5530458,unknown
2,232022460,unknown
3,232438397,2
4,230460314,unknown


In [8]:
df_classes.groupby('class').size()

class
1            4545
2           42019
unknown    157205
dtype: int64

In [9]:
tx_features = ["tx_feat_"+str(i) for i in range(2,95)]
agg_features = ["agg_feat_"+str(i) for i in range(1,73)]
df_features.columns = ["txId","time_step"] + tx_features + agg_features
features = pd.merge(df_features,df_classes,left_on="txId",right_on="txId",how='left')
features['class'] = features['class'].apply(lambda x: '0' if x == "unknown" else x)

The x-axis of the stacked bar chart likely represents the different time steps (time_step) in the data, while the y-axis represents the count of occurrences of each class ('Unknown', 'Licit', 'Illicit') within each time step.

In [10]:
count_by_class = features[["time_step",'class']].groupby(['time_step','class']).size().to_frame().reset_index()
illicit_count = count_by_class[count_by_class['class'] == '1']
licit_count = count_by_class[count_by_class['class'] == '2']
unknown_count = count_by_class[count_by_class['class'] == "0"]

In [13]:
x_list = list(range(1,50))
fig = go.Figure(data = [
    go.Bar(name="Unknown",x=x_list,y=unknown_count[0],marker = dict(color = 'rgba(120, 100, 180, 0.6)',
        line = dict(
            color = 'rgba(120, 100, 180, 1.0)',width=1))),
    go.Bar(name="Licit",x=x_list,y=licit_count[0],marker = dict(color = 'rgba(246, 78, 139, 0.6)',
        line = dict(
            color = 'rgba(246, 78, 139, 1.0)',width=1))),
    go.Bar(name="Illicit",x=x_list,y=illicit_count[0],marker = dict(color = 'rgba(58, 190, 120, 0.6)',
        line = dict(
            color = 'rgba(58, 190, 120, 1.0)',width=1)))

])
fig.update_layout(barmode='stack')
py.iplot(fig)